In [1]:
import time
import pyautogui as pg

In [25]:
pg.hotkey('alt','tab')
# pg.moveTo(200,80) #search bar
time.sleep(2)
pg.hotkey('ctrl', 'l')
time.sleep(3)
pg.moveTo(900,470) #login textbox
pg.click()
time.sleep(2)
pg.moveTo(900,560) #password textbox
pg.click()
time.sleep(2)

In [27]:
pg.hotkey('alt','tab')
time.sleep(5)

pg.hotkey("ctrl", 'l')
time.sleep(3)
pg.write(f"https://app.respond.io/user/login")
time.sleep(2)
pg.press("enter")

In [33]:
# export button

pg.hotkey('alt','tab')
time.sleep(2)
pg.moveTo(350,350)
time.sleep(3)

In [44]:
# download button
time.sleep(2)
pg.hotkey('alt','tab')
time.sleep(2)
pg.moveTo(1550,500)
time.sleep(5)

In [ ]:
#sign out button
time.sleep(2)
pg.hotkey('alt','tab')
time.sleep(1)
pg.moveTo(30,800)
pg.click()
time.sleep(2)
pg.moveTo(100,770)

In [36]:
import logging
import time
import pyautogui as pg
import imaplib
import email
import yaml
from email.header import decode_header
from datetime import datetime, timedelta, timezone

def load_creds(file_path):
    """Load credentials from YAML file"""
    try:
        with open(file_path, 'r') as file:
            creds = yaml.safe_load(file)
            logging.info("Successfully loaded credentials from YAML")
            return (
                creds.get('user'),
                creds.get('app-password')  # Using app-password instead of regular password
            )
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}", exc_info=True)
        raise

def check_for_export_email(creds_file, timeout_minutes=15):
    """
    Check Gmail for export completion email using credentials from YAML
    """
    logging.info("Starting to check for export completion email")
    
    try:
        # Load credentials
        email_address, app_password = load_creds(creds_file)
        
        # Connect to Gmail IMAP
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(email_address, app_password)
        logging.info("Successfully connected to Gmail")
        
        # Use timezone-aware datetime for start_time
        start_time = datetime.now(timezone.utc)
        while datetime.now(timezone.utc) - start_time < timedelta(minutes=timeout_minutes):
            # Select inbox
            mail.select("INBOX")
            
            # Search for emails with specific subject
            search_query = 'SUBJECT "Data export request completed for Customer Success | respond.io"'
            _, message_numbers = mail.search(None, search_query)
            
            if message_numbers[0]:
                # Get the latest email's date
                latest_email_id = message_numbers[0].split()[-1]
                _, msg_data = mail.fetch(latest_email_id, "(RFC822)")
                email_body = msg_data[0][1]
                message = email.message_from_bytes(email_body)
                date_str = message["date"]
                date = email.utils.parsedate_to_datetime(date_str)  # This returns timezone-aware datetime
                
                # Now both datetimes are timezone-aware
                if datetime.now(timezone.utc) - date < timedelta(minutes=15):
                    logging.info("Export completion email found")
                    mail.logout()
                    return True
            
            # Wait before checking again
            logging.info("Export email not found yet, waiting 30 seconds...")
            time.sleep(30)
        
        logging.warning(f"Email not found within {timeout_minutes} minutes timeout")
        mail.logout()
        return False
        
    except Exception as e:
        logging.error(f"Error checking email: {str(e)}", exc_info=True)
        return False
    
check_for_export_email('credentials.yaml')

True